---

4. 데이터 전처리

---

4.1 누락된 데이터 다루기

In [3]:
# 4.1.1누락된 값 식별

import pandas as pd
import numpy as np
from io import StringIO
# CSV파일의 줄바꿈 기호로 ...쓸 필요 없음
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
print('df:')
print(df)
print('')
print('df.isnull().sum():')
print(df.isnull().sum())
print('df.values:')
print(df.values)
print('')


df:
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN

df.isnull().sum():
A    0
B    0
C    1
D    1
dtype: int64
df.values:
[[ 1.  2.  3.  4.]
 [ 5.  6. nan  8.]
 [10. 11. 12. nan]]



In [4]:

# 4.1.2 누락된 데이터 제외
print('df.dropna(axis=0)는 행 삭제')
print(df.dropna(axis=0)) # axis=0은 행 삭제
print('df.dropna(axis=1)는 열 삭제')
print(df.dropna(axis=1)) # axis=1은 열 삭제
print('')


df.dropna(axis=0)는 행 삭제
     A    B    C    D
0  1.0  2.0  3.0  4.0
df.dropna(axis=1)는 열 삭제
      A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0



In [5]:

#4.1.3 누락된 값 대체
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan,strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
print("imputed_data")
print(imputed_data)
print('')
# .fillna(df.mean())을 이용해 쉽게 빈 데이터 처리
print('df.fillna(df.mean())',end='')
print(df.fillna(df.mean()))
#4.1.4 사이킷런 추정 api익히기
#데이터값 학습을 통한 추정


imputed_data
[[ 1.   2.   3.   4. ]
 [ 5.   6.   7.5  8. ]
 [10.  11.  12.   6. ]]

df.fillna(df.mean())      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   7.5  8.0
2  10.0  11.0  12.0  6.0


4.2 범주형 데이터 다루기

In [6]:
#4.2.1 판다스를 사용한 범주형 데이터 인코딩
import pandas as pd 
df= pd.DataFrame([
    ['green','M',10.1,'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue','XL',15.3,'class2']
])
df.columns = ['color','size','price','classlabel']


In [7]:

#4.2.2 순서가 있는 특성 매핑
size_mapping = {
    'XL':3,
    'L':2,
    'M':1
}
# inv_size_mapping = {v: k for k,v size_mapping.items()} #inverse mapping 함수
df['size'] = df['size'].map(size_mapping)
df['size']


0    1
1    2
2    3
Name: size, dtype: int64

In [8]:

#4.2.3 클래스 레이블 인코딩
#순서: mapping할 label들을 문장 안의 반복문을 사용해 딕셔너리를 만들고 map함수로 매핑함
import numpy as np

class_mapping={label:idx for idx,label in enumerate(np.unique(df['classlabel']))}
df['classlabel'] = df['classlabel'].map(class_mapping)


In [9]:

#4.2.4 순서가 없는 특성에 원-핫 인코딩 적용
#특성을 간단하게 추출, color 인코딩
from sklearn.preprocessing import LabelEncoder
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
print('LabelEncoder를 사용해 추출한 X')
print(X)
#원 핫 인코더 1개열 적용
from sklearn.preprocessing import OneHotEncoder
X = df[['color','size','price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:,0].reshape(-1,1))

#원 핫 인코더 다수 열 적용

#ColumTransformer사용
from sklearn.compose import ColumnTransformer
X = df[['color','size','price']].values

c_transf = ColumnTransformer([
    ('onehot',OneHotEncoder(dtype=int),[0]),
    ('noting','passthrough',[1,2])
])
c_transf.fit_transform(X)

#pandas get_dumies사용
print('')
print('pandas의 .get_dummies(column) 사용')
print(pd.get_dummies(df[['price','color','size']]))
print('첫 열 삭제')
print(pd.get_dummies(df[['price','color','size']],drop_first=True))
print('')

LabelEncoder를 사용해 추출한 X
[[1 1 10.1]
 [2 2 13.5]
 [0 3 15.3]]

pandas의 .get_dummies(column) 사용
   price  size  color_blue  color_green  color_red
0   10.1     1       False         True      False
1   13.5     2       False        False       True
2   15.3     3        True        False      False
첫 열 삭제
   price  size  color_green  color_red
0   10.1     1         True      False
1   13.5     2        False       True
2   15.3     3        False      False



4.3 데이터셋 나누기

In [10]:
#4.3 데이터셋 나누기
import pandas as pd
df_wine = pd.read_csv('/Users/youngki/study/Deep Learning/3.9.18/wine.data',header=None)
df_wine.columns = ['Class label','Alcohol','Malic acid','Ash','Alcalinity of ash',
                   'Magnesium', 'total phenols','Flavanoids','Nonflavanoid phenols',
                   'Proanthocyanins','Color intensity','Hue','OD280/OD315 of diluted wines','Proline']

df_wine.head() #head()는 앞의 5개 데이터 보여줌

from sklearn.model_selection import train_test_split

X,y = df_wine.iloc[:,1:].values, df_wine.iloc[:,0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X,y,test_size=0.3,random_state=0,stratify=y) # stratify는 y의 데이터의 비율 유지

4.4 특성스케일 맞추기

4.5 L1규제

In [12]:
from sklearn.linear_model import LogisticRegression
# C -> l1규제의 파라미터 규제정도와 정비례
lr= LogisticRegression(penalty='l1', C = 1.0, solver = 'libnlinear', multi_class = 'ovr')
